In [5]:
import requests
import pandas as pd
import numpy as np
import time
import json
import pandas_gbq
from google.oauth2.service_account import Credentials
from google.cloud import bigquery

# TODO: Set path to credentials 
cred_path = r'C:\Users\vicow\Documents\PlaceSense\credentials_cs.json'
credentials = Credentials.from_service_account_file(cred_path)

# Create GBQ client
#bqclient = bigquery.Client(credentials=credentials)

In [12]:
coordinates = pd.read_csv('coordinates/den_haag.csv')

locations = []
for i,j in zip(coordinates['avg_longitude'], coordinates['avg_latitude']):
    locations.append([i,j])
locations = locations[:10]

locations

[[4.378308747355544, 52.06905274939536],
 [4.289123111155115, 52.070427006374416],
 [4.277123128301878, 52.09660214513916],
 [4.286990047163461, 52.064168775563815],
 [4.280999322209033, 52.04303419899174],
 [4.29965745028815, 52.09077143804953],
 [4.267167041204614, 52.05151623343131],
 [4.298759130910276, 52.01527408010378],
 [4.365983846451943, 52.0955401845543],
 [4.279287121964956, 52.078123824787255]]

In [6]:
sql = """
    SELECT 
        level, country_1, entity_type_1, entity_code_1,
        ARRAY_AGG(entity_code_2) entity_code_2
    FROM `gifted-slice-218906.production.geolocation_distances`
    LEFT JOIN (SELECT level level_1, aerial_distance_km_filter FROM `gifted-slice-218906.production.geolocation_levels`) ON level_1 = level
    WHERE level = 1 AND aerial_distance < aerial_distance_km_filter
    GROUP BY level, country_1, entity_type_1, entity_code_1
    LIMIT 10000
"""

#geolocations = bqclient.query(sql).result().to_dataframe()
geolocations = pandas_gbq.read_gbq(sql, credentials=credentials)
geolocations.head()

Downloading: 100%|██████████| 10000/10000 [00:50<00:00, 198.36rows/s]


,level,country_1,entity_type_1,entity_code_1,entity_code_2
0,1,at,statistical_area,31026008,"[31321003, 32135002, 31104000, 31304000, 31111..."
1,1,at,statistical_area,31207003,"[91301110, 92301141, 31952004, 92301050, 92301..."
2,1,at,statistical_area,40101342,"[40802001, 41808002, 41807000, 40827000, 40831..."
3,1,at,statistical_area,30604002,"[92201131, 32131005, 92201134, 31921000, 92101..."
4,1,at,statistical_area,32305003,"[92301013, 92301093, 90501025, 90501029, 91201..."


In [7]:
sql = f"""
    SELECT entity_code entity_code_1, avg_longitude avg_longitude_1, avg_latitude avg_latitude_1 FROM `gifted-slice-218906.production.geolocations`
    WHERE entity_code in UNNEST({list(geolocations['entity_code_1'])})
"""

df = pandas_gbq.read_gbq(sql, credentials=credentials)
df.head()

Downloading: 100%|██████████| 11942/11942 [00:01<00:00, 9662.93rows/s]


,entity_code_1,avg_longitude_1,avg_latitude_1
0,BU03634803,4.855013,52.344482
1,BU19161200,4.353547,52.062187
2,BU19240003,3.982760,51.828038
3,BU04849404,4.612497,52.123073
4,BU05460404,4.496786,52.147876


In [8]:
geolocations = pd.merge(left=geolocations, right=df, how='left', on='entity_code_1')
geolocations.head()

,level,country_1,entity_type_1,entity_code_1,entity_code_2,avg_longitude_1,avg_latitude_1
0,1,at,statistical_area,31026008,"[31321003, 32135002, 31104000, 31304000, 31111...",15.858162,48.597422
1,1,at,statistical_area,31207003,"[91301110, 92301141, 31952004, 92301050, 92301...",16.417624,48.412800
2,1,at,statistical_area,40101342,"[40802001, 41808002, 41807000, 40827000, 40831...",14.313198,48.271267
3,1,at,statistical_area,30604002,"[92201131, 32131005, 92201134, 31921000, 92101...",16.238680,48.008278
4,1,at,statistical_area,32305003,"[92301013, 92301093, 90501025, 90501029, 91201...",16.266005,47.885716


In [13]:
locations = []
for i,j in zip(geolocations['avg_longitude_1'], geolocations['avg_latitude_1']):
    locations.append([i,j])
locations = locations[:1]

locations

[[15.858162118832068, 48.59742170205823]]

In [ ]:
body = {"locations":locations,"metrics":["distance", "duration"],"sources":[1]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624867d60a4709b54dbeb853215f4841061a',
    'Content-Type': 'application/json; charset=utf-8'
}

start = time.time()
call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=body, headers=headers)
result = json.loads(call.text)
end = time.time()

if call.status_code == 200:
    print("\nRequest successfull with response " + str(call.status_code))
    print("\nNumber of geolocations: " + str(len(locations)))
    print("Execution time: " + str(round(end-start, 3)) + " seconds")
    distances = np.array(result['distances'])
    print("Resulting matrix of shape: " + str(distances.shape))
else:
    print("\nRequest failed with response " + str(call.status_code))
    print("\nError: " + str(result['error']))